### Import

In [3]:
import numpy as np
import math
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from tqdm import tnrange, tqdm_notebook
from collections import Counter
from data_loader import DataLoader

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix 
import pickle


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [27]:
data_loader = DataLoader()
train, valid = data_loader.small_train_valid()
train, adversary = data_loader.small_train_valid()

loading data...
splitting data...
building vocabulary...
splitting data...
building vocabulary...


In [17]:
print('%d training examples' %len(train))
print('%d validation examples' %len(valid))


1250 training examples
1250 validation examples


## Naive Bayes - Unigram 

In [21]:
def build_dictionary(data, size):
    """
    Build word count dictionary from training data.
    Return the most common words list () based on input size 
    e.g. [('the', 15969), ('and', 8108), ('is', 5359)...], return type: list
    
    """
    dictionary = Counter()

    for i in range(len(data)):
        for word in data[i].text:
            dictionary[word] += 1
        
    return dictionary.most_common(size)

In [22]:
def extract_features(data, dictionary):
    
    """
    Build word count dictionary from training data.
    Return the most common words based on input size 
    """
    
    docID = 0
    features_matrix = np.zeros((len(data),len(dictionary)))
    label = []

    for i in range(len(data)):
        if data[i].label == ['pos']:
            label.append(1)
        else:
            label.append(0)
        
        for idx, d in enumerate(dictionary):
            wordID = idx
            features_matrix[docID, wordID] += data[i].text.count(d[0])
        docID = docID + 1
        
    return features_matrix, np.array(label)


In [41]:
uniNBdict = build_dictionary(train, 30000)

In [42]:
train_x, train_y = extract_features(train, uniNBdict)
test_x, test_y = extract_features(valid, uniNBdict)

In [43]:
uniNB = MultinomialNB()
uniNB.fit(train_x, train_y)

print("unigram - train dataset accuracy", uniNB.score(train_x, train_y))
print("unigram - valid dataset accuracy", uniNB.score(test_x, test_y))


unigram - train dataset accuracy 0.9792
unigram - valid dataset accuracy 0.7936


In [44]:
print(confusion_matrix(test_y, uniNB.predict(test_x)))

[[549  68]
 [190 443]]


## Naive Bayes - Bigrams

In [28]:
def bi_dictionary(data, size):
    
    """
    Build bigram count dictionary from training data.
    Return the most common words list () based on input size 
    e.g. [('of the', 1856), ('. STOP', 880), ('this movie', 753)...], return type: list
    
    """
    
    dictionary = Counter()

    for i in range(len(data)):
        
        bidata = data[i].text.copy()
        bidata.insert(0,"START")
        bidata.append("STOP")
        
        for j in range(len(bidata)-1):  
            bigrams = str(bidata[j]) + " " + str(bidata[j+1])
            dictionary[bigrams] += 1
    
    return dictionary.most_common(size)


In [29]:
def build_dict(text):
    
    """
    Build bigram count dictionary from text.
    e.g. {'the story': 1, 'thinking and': 1, 'this movie': 2 ... }, return type: dictionary
    """
    bi_dict = Counter()
    
    bitext = text.copy()
    bitext.insert(0,"START")
    bitext.append("STOP")
    
    for i in range(len(bitext) -1 ):
        bigrams = str(bitext[i]) + " " + str(bitext[i+1])
        bi_dict[bigrams] += 1
    
    return bi_dict

In [30]:
def bi_extract_features(data, dictionary):

    docID = 0
    features_matrix = np.zeros((len(data),len(dictionary)))
    label = []

    for i in range(len(data)):
        
        if data[i].label == ['pos']:
            label.append(1)
        else:
            label.append(0)
            
        bi_dict = {}
        bi_dict = build_dict(data[i].text)
        
        for idx, d in enumerate(dictionary):
            wordID = idx
            if d[0] in bi_dict:
                features_matrix[docID, wordID] += bi_dict[d[0]]
        docID = docID + 1
        
    return features_matrix, np.array(label)

In [45]:
biNB_dict = bi_dictionary(train, 30000)

In [46]:
train_x_bi, train_y_bi = bi_extract_features(train, biNB_dict)
test_x_bi, test_y_bi = bi_extract_features(valid, biNB_dict)

In [47]:
biNB = MultinomialNB()
biNB.fit(train_x_bi, train_y_bi)
print("bigram - train dataset accuracy", biNB.score(train_x_bi, train_y_bi))
print("bigram - valid dataset accuracy", biNB.score(test_x_bi, test_y_bi))

bigram - train dataset accuracy 0.9952
bigram - valid dataset accuracy 0.8168


In [48]:
print(confusion_matrix(test_y_bi, biNB.predict(test_x_bi)))

[[528  89]
 [140 493]]


## Adversary - swap word (black box)

In [52]:
# Because we perform one swap per word, but do not alter the first or last letters.
# This noise is only applied to words of length > 3.
# noise -> niose

def swap(word):
    rand = random.randint(1,len(word)-3)
    return word[:rand] +  word[rand:rand+2][::-1] + word[rand+2:]

def adversary_swap(adversary_data, propotion):
    """
    Randomly swap characters in words in text, parameter 'propotion' decides how many words are s
    e.g. noise -> niose 
    """
    for i in range(len(adversary_data)):
        rand_sample = random.sample(np.arange(0, len(adversary_data[i].text)).tolist(), int(len(adversary_data[i].text)*propotion))
        for pick in rand_sample:
            if len(adversary_data[i].text[pick]) > 3:
                #print(adversary_data[i].text[pick], "->", swap(adversary_data[i].text[pick]))
                adversary_data[i].text[pick] = swap(adversary_data[i].text[pick])
        #print(" ")
        #print(i, "SWAP END")
        #print(" ")
        
    return adversary_data


In [53]:
adversary_2 = adversary_swap(adversary, 0.8)

### Adversary - Unigram Naive Bayes

In [57]:
adver_x_uni, adver_y_uni = extract_features(adversary_2, dict_10k)
print("unigram - adversary dataset accuracy", uniNB.score(adver_x_uni, adver_y_uni))
print(confusion_matrix(test_y, uniNB.predict(test_x)))
print(confusion_matrix(adver_y_uni, uniNB.predict(adver_x_uni)))


unigram - adversary dataset accuracy 0.7264
[[549  68]
 [190 443]]
[[539  78]
 [264 369]]


### Adversary - Bigram Naive Bayes

In [14]:
adver_x_bi, adver_y_bi = bi_extract_features(adversary_2, bi_dict_10k)

In [55]:
print("bigram - adversary dataset accuracy", biNB.score(adver_x_bi, adver_y_bi))

bigram - adversary dataset accuracy 0.7392


In [56]:
print(confusion_matrix(adver_y_bi, biNB.predict(adver_x_bi)))

[[453 164]
 [162 471]]


In [49]:
# save model
filename = 'uniNB_model.sav'
pickle.dump(uniNB, open(filename, 'wb'))
  

In [50]:
filename = 'biNB_model.sav'
pickle.dump(biNB, open(filename, 'wb'))

In [54]:
# load the model

uniNB = pickle.load(open('uniNB_model.sav', 'rb'))
biNB = pickle.load(open('biNB_model.sav', 'rb'))



In [ ]:
len(data_loader.TEXT.vocab.itos)
pretrained_embeddings = data_loader.TEXT.vocab.vectors
len(pretrained_embeddings)